<a href="https://colab.research.google.com/github/Kjgwo/BigData/blob/main/BigData5%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BeautifulSoup

In [ ]:
import requests
import re
crawling_url = 'http://www.hanbit.co.kr/store/books/full_book_list.html'
response = requests.get(crawling_url)
text = response.text

In [ ]:
from bs4 import BeautifulSoup as bs

In [ ]:
soup = bs(text, 'html.parser')

In [ ]:
result = soup.find_all('tr')

In [ ]:
urls = []
titles = []
authors = []
dates = []
brands = []
prices = []

for item in result:
    it1 = item.find_all('a')
    it2 = item.find_all('td')
    for it in it1:
        url = it.get('href')
        if(url == None):
            continue
        url = "https://www.hanbit.co.kr" + url
        urls.append(url)

        brands.append(it2[0].string)
        titles.append(it2[1].string)
        authors.append(it2[2].string)
        dates.append(it2[3].string)
        prices.append(it2[4].string)

In [ ]:
import pandas as pd
df = pd.DataFrame(
    {
        "BRAND": brands,
        "TITLE": titles,
        "AUTHOR": authors,
        "DATE": dates,
        "PRICE": prices
    }
)
df

In [ ]:
import requests
crawling_url = 'http://www.kyobobook.co.kr/bestSellerNew/bestseller.laf?orderClick=d79'
response = requests.get(crawling_url)
text = response.text

In [ ]:
from bs4 import BeautifulSoup as bs

In [ ]:
soup = bs(text, 'html.parser')

In [ ]:
date = soup.find('h4', {'class': 'title_best_basic'}).find('small').text
print(date)

In [ ]:
bestseller = soup.find('ul', {'class': 'list_type01'})
bestseller

In [ ]:
bestseller_list = bestseller.findAll('div', {'class': 'detail'})
len(bestseller_list)

책 제목 찾기

In [ ]:
title_list = []

for b_detail in bestseller_list:
    temp = b_detail.find('div', {'class': 'title'})
    title = temp.find('strong').text
    title_list.append(title)

저자, 출판사, 출판 일시 찾기

In [ ]:
def no_space(text):
    text1 = re.sub("\nbsp: | &nbsp: | \n|\t|\r", '',  text )
    text2 = re.sub("\n\n", '', text1)
    return text2

In [ ]:
def get_item(text, idx):
    text2 = ""
    cnt = 0
    for t in text:
        if( (len(text2) == 0 & (t == " "))):
            continue
        if( t == "\n"):
            continue
        if( cnt == idx-1):
            if( t != "|"):
                text2 = text2 + t
        if( t == "|"):
            cnt += 1
        if( cnt == idx):
            break
    return text2

In [ ]:
author_list = []
publisher_list = []
date_list = []

for b_detail in bestseller_list:
    temp = b_detail.find('div', {'class': 'author'})
    temp2 = temp.get_text()
    output = re.sub(r'<.*?>', '', temp2)
    output = re.sub('저자', '', output)
    output = re.sub('더보기', '', output)
    output = no_space(output)
    author = get_item(output, 1)
    author_list.append(author)
    publisher = get_item(output, 2)
    publisher_list.append(publisher)
    date = get_item(output, 3)
    date_list.append(date)

가격 찾기

In [ ]:
price_list = []
for b_detail in bestseller_list:
    temp2 = b_detail.find('div', {'class': 'price'})
    price = temp2.find('strong').text1
    price_list.append(price)

In [ ]:
import pandas as pd
df3 = pd.DataFrame(
    {
        'TITLE': title_list,
     'AUTHOR': author_list,
    'DATE': date_list,
    'PRICE': price_list,
    'PUBLISHER': publisher_list
    }
)

In [ ]:
df3